In [1]:

import pandas as pd
from collections import defaultdict
from itertools import combinations

df = pd.read_csv("Groceries_dataset.csv")
baskets = df.groupby('Member_number')['itemDescription'].apply(set).tolist()
items = sorted(df['itemDescription'].unique())
thresh = 19.28

item_counts = defaultdict(int)
for basket in baskets:
    for item in basket:
        item_counts[item] += 1

frequent_items = {item for item, count in item_counts.items() if count >= thresh}
lstfreqitems = list(frequent_items)


pair_counts = defaultdict(int)
for basket in baskets:
    for pair in combinations(sorted(basket & frequent_items), 2):
        pair_counts[pair] += 1

freq_pairs = {pair for pair, count in pair_counts.items() if count >= thresh}
lstfreqpairs = list(freq_pairs)

triplet_counts = defaultdict(int)
for basket in baskets:
    basket_items = basket & frequent_items
    for triplet in combinations(sorted(basket_items), 3):
        if all((pair in freq_pairs) for pair in combinations(triplet, 2)):
            triplet_counts[triplet] += 1
freq_triplets = {triplet for triplet, count in triplet_counts.items() if count >= thresh}

print()
rules = []
for i in lstfreqitems:
    for j in lstfreqitems:
        if i != j:
            both = pair_counts[tuple(sorted(tuple((str(i), str(j)))))]/len(baskets)
            suppfirst = item_counts[i]/len(baskets)
            conf = both/suppfirst
            lift = both/((suppfirst*(item_counts[j]/len(baskets))))
            if conf >= .003 and lift >1:
                rules.append([i,j,round(conf,3), round(lift,3)])

for i in lstfreqpairs:
    for j in lstfreqitems:
        if j not in i:
            antecedent = triplet_counts[tuple(sorted(tuple(list(i)+[j])))]/len(baskets)
            consequent = pair_counts[tuple(sorted(i))]/len(baskets)
            conf = antecedent/consequent
            lift = antecedent/(consequent*(item_counts[j]/len(baskets)))
            if conf >= .003 and lift >1:
                rules.append([i,j, round(conf,3),round(lift,3)])
for i in range(len(rules)):
    print("Rule: {} implies {} with {} confidence and {} lift".format(rules[i][0], rules[i][1], rules[i][2], rules[i][3]))
    if i == 10:
        print("These are the first 10 rules")
        break





Rule: frankfurter implies pip fruit with 0.198 confidence and 1.159 lift
Rule: frankfurter implies berries with 0.08 confidence and 1.006 lift
Rule: frankfurter implies abrasive cleaner with 0.011 confidence and 1.983 lift
Rule: frankfurter implies napkins with 0.082 confidence and 1.009 lift
Rule: frankfurter implies domestic eggs with 0.162 confidence and 1.219 lift
Rule: frankfurter implies sparkling wine with 0.015 confidence and 1.265 lift
Rule: frankfurter implies fruit/vegetable juice with 0.142 confidence and 1.135 lift
Rule: frankfurter implies misc. beverages with 0.095 confidence and 1.613 lift
Rule: frankfurter implies cream cheese  with 0.095 confidence and 1.075 lift
Rule: frankfurter implies beverages with 0.069 confidence and 1.112 lift
Rule: frankfurter implies pickled vegetables with 0.037 confidence and 1.119 lift
These are the first 10 rules


The presence of antecedents implies the consequents above with a minimum confidence threshold of .3.

1. Support is just the frequency of an itemset in all of the baskets, so it measures how often something occurs or how probable it is to occur. Confidence is basically how related to items are to each other. It is like the ratio of how many times the items occur together normalized by the number of times the antecedent occurs. Lift is basically how related the antecedents and consequents are in an itemset normalized by their independance. If it is greater than 1 than that means there is a positive correlation while less than 1 means a negative correlation.

2. Support is the basis for most of our calculations to determine confidence, interestingness, and lift. We can make a lot of conclusions on relatedness of items based on how popular they are from this calculation, its like the base point for everything. Lift helps us filter out rules we are confident in that are in pairs based on their frequency. If we have a lot of pairs that reach a minimum support, we only want the ones that we are a certain level of confident about, meaning they occur frequently together compared to seperately. It doesn't really mean anything if a pair reached the support but they only occur one time together compared to the items occuring 20 times seperately, its more of a coincidence compared to confidence that they are correlated to each other. It measures the observed confidence compared to what was expected, to suggest there was a positive or negative correlation if it occured more or less than just random chance or what was expected. Lift requires that we know the confidence about the antecedents and the consequents, it helps determine how confident we are that there is a rule that has meaning. This also helps us filter out frequent pairs if they occure frequently together compared to how frequently the antecedent occurs. It measures the strength of two itemsets. Association rules are created by keeping track of items support, which we can build off of from monotonicity property to keep track of frequent pairs and frequent triplets and so on. Once we have frequent pairs we can try to start to find association rules only with a certain level of confidence though. We can check how often does one occur in the pair in relation to the other, and then how confident are we in this. We only want the ones we are confident in to a certain level because we want them to have a certain strength of association. Lastly, once we find association rules, we can basically say with a certain confidence level or positive/negative correlation that if we are going to put this item or these two items next to each other, we would benefit by putting the item it would imply next to it, because people like to buy them together. We could also use it to say we shouldn't put these together because people don't usually buy these things together and that wouldn't do us any good. We can also put emphasis on the ones that have a high confidence level so we know there is a greater chance they are going to buy those things together. I only outputted the rules that had a lift of greater than one to make sure there was a positive correlation and the observed support was greater than what was expected of them independently to filter some of the rules out since there are so many.

In [2]:
from collections import defaultdict
from itertools import combinations
import itertools
def PCY(transaction, supp_threshold, conf_threshold):
    item_counts = defaultdict(int)
    hash_table = defaultdict(int)
    pairs = defaultdict(int)
    for basket in transaction:
        for item in basket:
            item_counts[item] += 1
        for i in range(len(basket)):
            for j in range(i+1, len(basket)):
                hash_table[hash((basket[i], basket[j]))%10007] += 1
    freqitems = {item: count for item, count in item_counts.items() if count >= 19.28}
    bitmap = {bucket: 1 if count >= 19.38 else 0 for bucket, count in hash_table.items()}

    candidate_pairs = defaultdict(int)

    for basket in transaction:
        for i in range(len(basket)):
          for j in range(i+1, len(basket)):
            pair = (basket[i], basket[j])
            hashed_pair = hash(pair)%10007
            if bitmap[hashed_pair] == 1 and pair[0] in freqitems and pair[1] in freqitems:
                candidate_pairs[pair] += 1

    frequentpairs = {tuple(sorted(pair)):count for pair, count in candidate_pairs.items() if count >= 19.28}

    rulespcy = []
    for i in freqitems:
      for j in freqitems:
        if i != j:
            pair = tuple(sorted(tuple((str(i), str(j)))))
            if pair in frequentpairs:
              both = frequentpairs[tuple(sorted(tuple((str(i), str(j)))))]/len(baskets)
              suppfirst = freqitems[i]/len(baskets)
              conf = both/suppfirst
              lift = both/((suppfirst*(freqitems[j]/len(baskets))))
              if conf >= .003 and lift >1:
                rulespcy.append([i,j,round(conf,3), round(lift,3)])
    return rulespcy, freqitems, frequentpairs


items = df.groupby('Member_number')['itemDescription'].apply(list).reset_index()
items = items['itemDescription'].tolist()
for i in range(len(PCY(items, .0005, .003)[0])):
    print("Rule: {} implies {} with {} confidence and {} lift".format(rules[i][0], rules[i][1], rules[i][2], rules[i][3]))
    if i == 10:
      print("These are the first 10 rules")
      break

print("Freqeunt Items")

print(PCY(items, .0005, .003)[1])

print("Frequent Pairs")

print(PCY(items, .0005, .003)[2])




Rule: frankfurter implies pip fruit with 0.198 confidence and 1.159 lift
Rule: frankfurter implies berries with 0.08 confidence and 1.006 lift
Rule: frankfurter implies abrasive cleaner with 0.011 confidence and 1.983 lift
Rule: frankfurter implies napkins with 0.082 confidence and 1.009 lift
Rule: frankfurter implies domestic eggs with 0.162 confidence and 1.219 lift
Rule: frankfurter implies sparkling wine with 0.015 confidence and 1.265 lift
Rule: frankfurter implies fruit/vegetable juice with 0.142 confidence and 1.135 lift
Rule: frankfurter implies misc. beverages with 0.095 confidence and 1.613 lift
Rule: frankfurter implies cream cheese  with 0.095 confidence and 1.075 lift
Rule: frankfurter implies beverages with 0.069 confidence and 1.112 lift
Rule: frankfurter implies pickled vegetables with 0.037 confidence and 1.119 lift
These are the first 10 rules
Freqeunt Items
{'soda': 1514, 'canned beer': 717, 'sausage': 924, 'whole milk': 2502, 'pickled vegetables': 134, 'misc. bevera

To compare the PCY algorithm with the Apriori algorithm, they both found the same amount of frequent items and frequent pairs. The PCY algorithm ran faster because it is less computationally complex, and would run faster for larger sets of data. We also only learned 2 passes for the PCY algorithm in class instead of 3 for the apriori algorithm. I have printed all of the rules I produced as well as the frequent items and frequent pairs. I was not able to produce the same amount of association rules since we only learned two passes in class, so to create association rules with two items in the antecedent you would need a third pass to find all of the triplets to check the support of the antecedent.